In [4]:
!pip install --upgrade pip


  Using cached pip-24.0-py3-none-any.whl (2.1 MB)


In [5]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-sko9nb7w
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-sko9nb7w
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Requirement already up-to-date: accelerate in /home/gautam/.local/lib/python3.8/site-packages (0.28.0)
Requirement already up-to-date: datasets[audio] in /home/gautam/.local/lib/python3.8/site-packages (2.18.0)
  Created wheel for transformers: filename=transformers-4.40.0.dev0-py3-none-any.whl size=8796955 sha256=e32b089c4baf245790ce8ff4de42275f49da938767651d14205a8542e6362b9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-mbvs0tcc/wheels/05/0a/97/64ae47c27ba95fae2cb5838e7b4b7247a34d4a8ba5f7092de2
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0.dev0
    Uninstalling transformers-4.40.0.dev0:
 

In [6]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


/home/gautam/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

In [8]:

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)

In [ ]:




model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])
